# Sentiment Analysis

Model: [cardiffnlp/twitter-roberta-base-sentiment-latest](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)

**Important: This notebook must be placed in folder `/transfomers`**


## AI Lab config

In [1]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

## Get Model

In [15]:
MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

In [16]:
%run utils/model_retrieval.ipynb
load_huggingface_model(ai_lab_config, MODEL_NAME, 'sequence_classification')

## Invoke model with UDF

In [20]:
MY_TEXT = "I'm having an issue with the Bose Sound Link. Please assist. This problem started occurring after the recent software update. I haven't made any other changes to the device."

# Make sure our texts can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

print(MY_TEXT)

I''m having an issue with the Bose Sound Link. Please assist. This problem started occurring after the recent software update. I haven''t made any other changes to the device.


In [21]:
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.nb_connector.language_container_activation import get_activation_sql

# We will be creating a new database session every time we call the model.
# We will have to activate the language container for each of these sessions.
# Here we will get the activation SQL command.
activation_sql = get_activation_sql(ai_lab_config)

In [28]:
sql = f"""
SELECT {ai_lab_config.db_schema}.TE_SEQUENCE_CLASSIFICATION_SINGLE_TEXT_UDF(
    NULL,
    '{ai_lab_config.te_bfs_connection}',
    '{ai_lab_config.te_models_bfs_dir}',
    '{MODEL_NAME}',
    '{MY_TEXT}'
)
"""

print(activation_sql)
print(sql)

with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(query=activation_sql)
    result = conn.export_to_pandas(query_or_table=sql)    

# Show result data frame
result

ALTER SESSION SET SCRIPT_LANGUAGES='R=builtin_r JAVA=builtin_java PYTHON3=builtin_python3 PYTHON3_TE=localzmq+protobuf:///uploads/default/TE/exasol_transformers_extension_container_release?lang=python#/buckets/uploads/default/TE/exasol_transformers_extension_container_release/exaudf/exaudfclient_py3';

SELECT MN_AILAB.TE_SEQUENCE_CLASSIFICATION_SINGLE_TEXT_UDF(
    NULL,
    'TE_BFS_EXASOL_MN',
    'te_models',
    'cardiffnlp/twitter-roberta-base-sentiment-latest',
    'I''m having an issue with the Bose Sound Link. Please assist. This problem started occurring after the recent software update. I haven''t made any other changes to the device.'
)



,BUCKETFS_CONN,SUB_DIR,MODEL_NAME,TEXT_DATA,LABEL,SCORE,ERROR_MESSAGE
0,TE_BFS_EXASOL_MN,te_models,cardiffnlp/twitter-roberta-base-sentiment-latest,I'm having an issue with the Bose Sound Link. ...,negative,0.828092,NaN
1,TE_BFS_EXASOL_MN,te_models,cardiffnlp/twitter-roberta-base-sentiment-latest,I'm having an issue with the Bose Sound Link. ...,neutral,0.163371,NaN
2,TE_BFS_EXASOL_MN,te_models,cardiffnlp/twitter-roberta-base-sentiment-latest,I'm having an issue with the Bose Sound Link. ...,positive,0.008537,NaN
